**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Use el siguiente conjunto de datos para resolver los problemas propuestos. Use [gradetool](gradetool.md) para verificar las respuestas del notebook.

In [1]:
%%writefile data.csv
1,Vivian,Hamilton,1971-07-08,green,1
2,Karen,Holcomb,1974-05-23,green,4
3,Cody,Garrett,1973-04-22,orange,1
4,Roth,Fry,1975-01-29,black,1
5,Zoe,Conway,1974-07-03,blue,2
6,Gretchen,Kinney,1974-10-18,viole,1
7,Driscoll,Klein,1970-10-05,blue,5
8,Karyn,Diaz,1969-02-24,red,1
9,Merritt,Guy,1974-10-17,indigo,4
10,Kylan,Sexton,1975-02-28,black,4
11,Jordan,Estes,1969-12-07,indigo,4
12,Hope,Coffey,1973-12-24,green,5
13,Vivian,Crane,1970-08-27,gray,5
14,Clio,Noel,1972-12-12,red,5
15,Hope,Silva,1970-07-01,blue,5
16,Ayanna,Jarvis,1974-02-11,orange,5
17,Chanda,Boyer,1973-04-01,green,4
18,Chadwick,Knight,1973-04-29,yellow,1

Writing data.csv


In [2]:
%load_ext bigdata
%pig_init

In [3]:
%%pig
u = LOAD 'data.csv' USING PigStorage(',') 
    AS (id:int, 
        firstname:CHARARRAY, 
        surname:CHARARRAY, 
        birthday:CHARARRAY, 
        color:CHARARRAY, 
        quantity:INT);

## Pregunta 1

Escriba una consulta en Pig que genere la siguiente salida:

    (Vivian@Hamilton)
    (Karen@Holcomb)
    (Cody@Garrett)
    (Roth@Fry)
    (Zoe@Conway)
    (Gretchen@Kinney)
    (Driscoll@Klein)
    (Karyn@Diaz)
    (Merritt@Guy)
    (Kylan@Sexton)
    (Jordan@Estes)
    (Hope@Coffey)
    (Vivian@Crane)
    (Clio@Noel)
    (Hope@Silva)
    (Ayanna@Jarvis)
    (Chanda@Boyer)
    (Chadwick@Knight)

In [4]:
%%pig
nombre = foreach u Generate CONCAT (firstname,'@',surname); 
dump nombre;

(Vivian@Hamilton)
(Karen@Holcomb)
(Cody@Garrett)
(Roth@Fry)
(Zoe@Conway)
(Gretchen@Kinney)
(Driscoll@Klein)
(Karyn@Diaz)
(Merritt@Guy)
(Kylan@Sexton)
(Jordan@Estes)
(Hope@Coffey)
(Vivian@Crane)
(Clio@Noel)
(Hope@Silva)
(Ayanna@Jarvis)
(Chanda@Boyer)
(Chadwick@Knight)


## Pregunta 2

Genere una relación con el apellido y su longitud. Ordene por longitud y por apellido. Obtenga la siguiente salida.

    (Hamilton,8)
    (Garrett,7)
    (Holcomb,7)
    (Coffey,6)
    (Conway,6)

In [5]:
%%pig
size = foreach u Generate surname,SIZE(surname) as x;
size_org = ORDER size BY x desc,surname asc;
lim_5 = LIMIT size_org 5;
DUMP lim_5;

(Hamilton,8)
(Garrett,7)
(Holcomb,7)
(Coffey,6)
(Conway,6)


## Pregunta 3

Escriba el código que genere la siguiente salida.

    (Hamilton,HAMILTON,hamilton)
    (Holcomb,HOLCOMB,holcomb)
    (Garrett,GARRETT,garrett)
    (Fry,FRY,fry)
    (Conway,CONWAY,conway)

In [6]:
%%pig
table = foreach u Generate surname,UPPER(surname),LOWER(surname);
lim_5 = LIMIT table 5;
dump lim_5;

(Hamilton,HAMILTON,hamilton)
(Holcomb,HOLCOMB,holcomb)
(Garrett,GARRETT,garrett)
(Fry,FRY,fry)
(Conway,CONWAY,conway)


## Pregunta 4

Obtenga los apellidos que empiecen por las letras entre la 'd' y la 'k'. La salida esperada es la siguiente:

    (Hamilton)
    (Holcomb)
    (Garrett)
    (Fry)
    (Kinney)
    (Klein)
    (Diaz)
    (Guy)
    (Estes)
    (Jarvis)
    (Knight)

In [7]:
%%pig
x = foreach u Generate surname; 
j = FILTER x by UPPER(SUBSTRING(surname,0,1)) MATCHES 'D|E|F|G|H|I|J|K';
dump j;

(Hamilton)
(Holcomb)
(Garrett)
(Fry)
(Kinney)
(Klein)
(Diaz)
(Guy)
(Estes)
(Jarvis)
(Knight)


## Pregunta 5

Escriba el código equivalente a la siguiente consulta en SQL.

    SELECT
        color
    FROM 
        u 
    WHERE 
        color 
    LIKE 'b%';

In [8]:
%%pig
data = foreach u Generate color;
lim = FILTER data by LOWER(SUBSTRING(color,0,1)) MATCHES 'b';
dump lim;

(black)
(blue)
(blue)
(black)
(blue)


## Pregunta 6

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        color 
    FROM 
        u 
    WHERE 
        color NOT LIKE 'b%';

In [9]:
%%pig
data = foreach u Generate color;
fil = FILTER data by not LOWER(SUBSTRING(color,0,1)) MATCHES  'b';
dump fil;

(green)
(green)
(orange)
(viole)
(red)
(indigo)
(indigo)
(green)
(gray)
(red)
(orange)
(green)
(yellow)


## Pregunta 7

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u 
    WHERE color = 'blue' AND firstname LIKE 'Z%';

In [10]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by UPPER(SUBSTRING(firstname,0,1)) MATCHES 'Z';
data_b = FILTER data_a by color MATCHES 'blue';
dump data_b;

(Zoe,blue)


## Pregunta 8

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u
    WHERE color = 'blue' OR firstname LIKE 'K%';

In [11]:
%%pig
t = foreach u Generate firstname,color;
a = FILTER t by (color=='blue' or UPPER(SUBSTRING(firstname,0,1))=='K');
dump a;

(Karen,green)
(Zoe,blue)
(Driscoll,blue)
(Karyn,red)
(Kylan,black)
(Hope,blue)


## Pregunta 9

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, color 
    FROM 
        u
    WHERE color IN ('blue','black');

In [12]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES 'blue|black';
dump data_a;

(Roth,black)
(Zoe,blue)
(Driscoll,blue)
(Kylan,black)
(Hope,blue)


## Pregunta 10

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, 
        color 
    FROM 
        u 
    WHERE color NOT IN ('blue','black');

In [13]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by not color MATCHES 'blue|black';
dump data_a;

(Vivian,green)
(Karen,green)
(Cody,orange)
(Gretchen,viole)
(Karyn,red)
(Merritt,indigo)
(Jordan,indigo)
(Hope,green)
(Vivian,gray)
(Clio,red)
(Ayanna,orange)
(Chanda,green)
(Chadwick,yellow)


## Pregunta 11

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u 
    WHERE 
        color REGEXP '^b';

In [14]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES '.*^b.*';
dump data_a;

(Roth,black)
(Zoe,blue)
(Driscoll,blue)
(Kylan,black)
(Hope,blue)


## Pregunta 12

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT
        firstname,
        color
    FROM 
        u 
    WHERE 
        color REGEXP '^[^b]';

In [15]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES '.*^[^b].*';
dump data_a;

(Vivian,green)
(Karen,green)
(Cody,orange)
(Gretchen,viole)
(Karyn,red)
(Merritt,indigo)
(Jordan,indigo)
(Hope,green)
(Vivian,gray)
(Clio,red)
(Ayanna,orange)
(Chanda,green)
(Chadwick,yellow)


## Pregunta 13

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u
    WHERE 
        color REGEXP 'blue|green';

In [16]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES '.*blue|green.*';
dump data_a;

(Vivian,green)
(Karen,green)
(Zoe,blue)
(Driscoll,blue)
(Hope,green)
(Hope,blue)
(Chanda,green)


## Pregunta 14

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color
    FROM 
        u
    WHERE 
        color REGEXP '.n';

In [17]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES '.*n$.*';
dump data_a;

(Vivian,green)
(Karen,green)
(Hope,green)
(Chanda,green)


## Pregunta 15

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname,
        color 
    FROM 
        u 
    WHERE 
        color REGEXP '[aeiou]$';

In [18]:
%%pig
data = foreach u Generate firstname,color;
data_a = FILTER data by color MATCHES '.*[aeiou]$.*';
dump data_a;

(Cody,orange)
(Zoe,blue)
(Gretchen,viole)
(Driscoll,blue)
(Merritt,indigo)
(Jordan,indigo)
(Hope,blue)
(Ayanna,orange)


## Pregunta 16

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        REGEX_EXTRACT(birthday, '....-..-..', 2) 
    FROM 
        u;

In [19]:
%%pig
data = foreach u Generate SUBSTRING(birthday,5,7);
dump data;

(07)
(05)
(04)
(01)
(07)
(10)
(10)
(02)
(10)
(02)
(12)
(12)
(08)
(12)
(07)
(02)
(04)
(04)


## Pregunta 17

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT  
        firstname,
        SUBSTRING_INDEX(firstname, 'a', 1)
    FROM 
        u;

In [20]:
%%pig
data = foreach u Generate INDEXOF(firstname,'a',0);
dump data;

(4)
(1)
(-1)
(-1)
(-1)
(-1)
(-1)
(1)
(-1)
(3)
(4)
(-1)
(4)
(-1)
(-1)
(2)
(2)
(2)


## Pregunta 18

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname 
    FROM 
        u 
    WHERE 
        SUBSTRING(firstname, 0, 1) >= 'm';

In [21]:
%%pig
data = foreach u Generate firstname;
data_a = FILTER data by UPPER(SUBSTRING(firstname,0,1)) >= 'M';
dump data_a;

(Vivian)
(Roth)
(Zoe)
(Merritt)
(Vivian)


## Pregunta 19

Escriba el código equivalente a la siguiente consulta SQL.

    SELECT 
        firstname, 
        LOCATE('ia', firstname) 
    FROM 
        u;

In [22]:
%%pig
data = foreach u Generate INDEXOF(firstname,'ia',0);
dump data;

(3)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(-1)
(3)
(-1)
(-1)
(-1)
(-1)
(-1)


## Pregunta 20

Escriba el código en Pig para manipulación de fechas que genere sea equivalente a la siguiente consulta SQL.

    SELECT 
        birthday, 
        DATE_FORMAT(birthday, "yyyy"),
        DATE_FORMAT(birthday, "yy"),
    FROM 
        persons
    LIMIT
        5;

In [23]:
%%pig
data = FOREACH u 
GENERATE SUBSTRING(birthday,0,4), SUBSTRING(birthday,2,4);
Dump data;

(1971,71)
(1974,74)
(1973,73)
(1975,75)
(1974,74)
(1974,74)
(1970,70)
(1969,69)
(1974,74)
(1975,75)
(1969,69)
(1973,73)
(1970,70)
(1972,72)
(1970,70)
(1974,74)
(1973,73)
(1973,73)


## Pregunta 21

Escriba el código en Pig para manipulación de fechas que genere la siguiente salida.

    (1971-07-08,jul,07,7)
    (1974-05-23,may,05,5)
    (1973-04-22,abr,04,4)
    (1975-01-29,ene,01,1)
    (1974-07-03,jul,07,7)
    (1974-10-18,oct,10,10)
    (1970-10-05,oct,10,10)
    (1969-02-24,feb,02,2)
    (1974-10-17,oct,10,10)
    (1975-02-28,feb,02,2)
    (1969-12-07,dic,12,12)
    (1973-12-24,dic,12,12)
    (1970-08-27,ago,08,8)
    (1972-12-12,dic,12,12)
    (1970-07-01,jul,07,7)
    (1974-02-11,feb,02,2)
    (1973-04-01,abr,04,4)
    (1973-04-29,abr,04,4)

In [24]:
%%pig
data = FOREACH u GENERATE birthday, 
SUBSTRING(birthday,5,7) as m1, 
SUBSTRING(birthday,5,7) as m2;
data_a = FOREACH data GENERATE birthday,
(int)m1, 
m2;
data_b = FOREACH data_a GENERATE birthday,
(CASE m1 WHEN 1 THEN 'ene'  
           WHEN 2 THEN 'feb'  
           WHEN 3 THEN 'mar' 
           WHEN 4 THEN 'abr'  
           WHEN 5 THEN 'may'  
           WHEN 6 THEN 'jun'  
           WHEN 7 THEN 'jul'  
           WHEN 8 THEN 'ago'  
           WHEN 9 THEN 'sep'  
           WHEN 10 THEN 'oct'  
           WHEN 11 THEN 'nov'  
           WHEN 12 THEN 'dic' ELSE ''  END),
m2, 
m1;
DUMP data_b;

(1971-07-08,jul,07,7)
(1974-05-23,may,05,5)
(1973-04-22,abr,04,4)
(1975-01-29,ene,01,1)
(1974-07-03,jul,07,7)
(1974-10-18,oct,10,10)
(1970-10-05,oct,10,10)
(1969-02-24,feb,02,2)
(1974-10-17,oct,10,10)
(1975-02-28,feb,02,2)
(1969-12-07,dic,12,12)
(1973-12-24,dic,12,12)
(1970-08-27,ago,08,8)
(1972-12-12,dic,12,12)
(1970-07-01,jul,07,7)
(1974-02-11,feb,02,2)
(1973-04-01,abr,04,4)
(1973-04-29,abr,04,4)


## Pregunta 22

Escriba el código en Pig para manipulación de fechas que genere la siguiente salida:

    (1971-07-08,08,8,jue,jueves)
    (1974-05-23,23,23,jue,jueves)
    (1973-04-22,22,22,dom,domingo)
    (1975-01-29,29,29,mié,miércoles)
    (1974-07-03,03,3,mié,miércoles)
    (1974-10-18,18,18,vie,viernes)
    (1970-10-05,05,5,lun,lunes)
    (1969-02-24,24,24,lun,lunes)
    (1974-10-17,17,17,jue,jueves)
    (1975-02-28,28,28,vie,viernes)
    (1969-12-07,07,7,dom,domingo)
    (1973-12-24,24,24,lun,lunes)
    (1970-08-27,27,27,jue,jueves)
    (1972-12-12,12,12,mar,martes)
    (1970-07-01,01,1,mié,miércoles)
    (1974-02-11,11,11,lun,lunes)
    (1973-04-01,01,1,dom,domingo)
    (1973-04-29,29,29,dom,domingo)

In [25]:
%%pig
x = FOREACH u GENERATE birthday,
    ToDate(birthday,'yyyy-MM-dd') as date;
y = FOREACH x GENERATE birthday,LOWER(ToString(date,'dd')) as DD
    ,ToString(date,'d') as D,ToString(date,'E') as E,
    ToString(date,'EEEE') as EEEE;
DUMP y;

(1971-07-08,08,8,jue,jueves)
(1974-05-23,23,23,jue,jueves)
(1973-04-22,22,22,dom,domingo)
(1975-01-29,29,29,mié,miércoles)
(1974-07-03,03,3,mié,miércoles)
(1974-10-18,18,18,vie,viernes)
(1970-10-05,05,5,lun,lunes)
(1969-02-24,24,24,lun,lunes)
(1974-10-17,17,17,jue,jueves)
(1975-02-28,28,28,vie,viernes)
(1969-12-07,07,7,dom,domingo)
(1973-12-24,24,24,lun,lunes)
(1970-08-27,27,27,jue,jueves)
(1972-12-12,12,12,mar,martes)
(1970-07-01,01,1,mié,miércoles)
(1974-02-11,11,11,lun,lunes)
(1973-04-01,01,1,dom,domingo)
(1973-04-29,29,29,dom,domingo)

## Pregunta 23

Cuente la cantidad de personas nacidas por año.

In [26]:
%%pig
F = FOREACH u GENERATE birthday,SUBSTRING(birthday,0,4) as mes1;
B = GROUP F BY (mes1);
C= FOREACH B GENERATE group,COUNT(F.mes1) AS Total;
DUMP C

(1969,2)
(1970,3)
(1971,1)
(1972,1)
(1973,4)
(1974,5)
(1975,2)


---

In [27]:
!rm pig_* data.csv

rm: cannot remove 'pig_*': No such file or directory


---